In [2]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# import lightgbm as lgb

In [3]:
path = "../data/de_train.parquet"
train_df = pd.read_parquet(path,engine="pyarrow")
train_df.head(2)

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887


In [24]:
path = '../data/id_map.csv'
df_id_map = pd.read_csv(path)
print(df_id_map.shape)

(255, 3)


In [25]:
df_id_map.head(2)

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)


In [14]:
n_components = 35
n_components_for_compound_encoding = 20


# predict_method = 'train_aggregation_by_compounds'
# predict_method = 'train_aggregation_by_compounds_with_denoising_pca'
# predict_method = 'train_aggregation_by_compounds_with_denoising_ICA'
predict_method = 'train_aggregation_by_compounds_with_denoising_TSVD'

if '_pca' in predict_method:
    str_inf_target_dimred = 'PCA' 
    reducer = PCA(n_components=n_components )
elif '_ICA' in predict_method:
    str_inf_target_dimred = 'ICA' 
#     reducer = PCA(n_components=n_components )
    reducer = FastICA(n_components=n_components, random_state=0, whiten='unit-variance')
elif '_TSVD' in predict_method:
    str_inf_target_dimred = 'TSVD' 
#     reducer = PCA(n_components=n_components )
#     reducer = FastICA(n_components=n_components, random_state=0, whiten='unit-variance')
    reducer = TruncatedSVD(n_components=n_components, n_iter=7, random_state=42)
else:
    str_inf_target_dimred = ''
    
print(str_inf_target_dimred, reducer)

TSVD TruncatedSVD(n_components=35, n_iter=7, random_state=42)


In [15]:
Y = train_df.iloc[:,5:].values
Yr = reducer.fit_transform(Y)

In [16]:
train_df.shape

(614, 18216)

In [17]:
Yr.shape

(614, 35)

In [18]:
Yr[:, :n_components_for_compound_encoding]

array([[ 2.89373825e+01,  3.25693174e+00, -2.17718795e+00, ...,
         1.71697383e+00,  2.43747294e+00,  1.17374369e+00],
       [ 9.34854576e+00,  8.66256211e+00, -2.06019648e+00, ...,
         9.57226672e-01,  4.68278086e+00, -3.77149246e+00],
       [-3.58110979e+01,  6.78206995e+00, -4.38171568e+00, ...,
         2.03040417e+00, -1.03710292e+00, -3.52523495e+00],
       ...,
       [-5.03158567e+00,  6.43942181e+00, -2.87129303e-04, ...,
         4.14887610e+00,  5.19326099e+00, -3.74356351e-01],
       [-5.28922641e+01,  7.50236438e+00, -2.89089984e+00, ...,
         1.12076210e+00,  1.74396997e+00, -1.35842003e+00],
       [-6.67468860e+01,  8.28862864e+00, -4.76962697e+00, ...,
        -5.42812517e+00,  2.95258313e+00, -2.17275842e+00]])

In [19]:
df_tmp = pd.DataFrame(Yr[:, :n_components_for_compound_encoding], index = train_df.index  )
df_tmp['column for aggregation'] = train_df['sm_name']
df_tmp.head(2)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,column for aggregation
0,28.937383,3.256932,-2.177188,0.468545,-5.621313,4.551657,0.579008,-1.745717,-0.093655,-5.538261,...,1.484589,1.648768,-1.687988,-0.827857,0.020230,-3.380221,1.716974,2.437473,1.173744,Clotrimazole
1,9.348546,8.662562,-2.060196,0.569955,-2.757469,5.832881,-3.394118,-3.942866,0.464306,-5.546544,...,-3.477309,0.317478,-1.808493,-0.951530,-0.743172,-4.011046,0.957227,4.682781,-3.771492,Clotrimazole


In [20]:
df_compound_encoded = df_tmp.groupby('column for aggregation').mean() # quantile( quantile )
print('df_compound_encoded.shape', df_compound_encoded.shape )

df_compound_encoded.shape (146, 20)


In [21]:
df_compound_encoded.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
column for aggregation,,,,,,,,,,,,,,,,,,,,
5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine,63.586319,-1.465497,-0.562572,-1.732934,20.996600,34.568289,-6.739745,-3.576149,0.356735,-25.661156,-2.904752,-4.330203,14.512692,2.759831,-0.440801,-1.129141,-3.808963,12.068773,10.358412,-1.580439
ABT-199 (GDC-0199),21.006970,1.129267,0.303375,-3.547908,-2.140733,-5.492110,2.184463,-0.790178,-0.359174,0.998044,0.166214,-0.462383,-1.396626,2.500967,-1.491096,-1.264126,1.647789,2.790226,1.024438,-0.689964


In [48]:
X = pd.DataFrame(index = train_df['sm_name']) # 
X['IX'] = np.arange(len(train_df))
X = X.join( df_compound_encoded , how = 'left').sort_values('IX')
X = X.iloc[:,1:]
X = X.values 
print( X.shape )
#X.head(3)

(614, 20)


In [49]:
X_submit = pd.DataFrame(index = df_id_map['sm_name']) # 
X_submit['IX'] = np.arange(len(X_submit))
X_submit = X_submit.join( df_compound_encoded , how = 'left').sort_values('IX')
X_submit = X_submit.iloc[:,1:]
X_submit = X_submit.values 
print( X_submit.shape )

(255, 20)


In [50]:
train_df.head(3)

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365


In [56]:
n_splits = 2
kf = KFold(n_splits=n_splits, random_state = 42, shuffle = True )
IX = pd.DataFrame(); IX['IX'] = range(len(train_df))

alpha_regularization_for_linear_models = 100000
model = Ridge(alpha=alpha_regularization_for_linear_models)

params = {
    'num_leaves': 5,
    'learning_rate': 0.1,
    'verbose': -1,
    'num_boost_round': 20,    # Number of boosting iterations (trees)
}

# model = MultiOutputRegressor( lgb.LGBMRegressor( **params) )
model =  MultiOutputRegressor( make_pipeline(StandardScaler(), SVR(C=20, epsilon=0.3, kernel = 'linear' ) ) )


Y = train_df.iloc[:,5:].values

Y_reduced_submit = np.zeros( (len(df_id_map) , Yr.shape[1] )   ); 
cnt_blend_submit = 0
Y_submit = np.zeros( (len(df_id_map) , 18211 )   ); 
cnt_blend_submit = 0

print(X.shape)
for i, (train_index, test_index) in enumerate(kf.split(IX)):
    print(f"Fold {i}:", len(test_index) )
    # print(f"  Train: index={train_index}")
    # print(f"  Test:  index={test_index}")
    
    Yr_train = reducer.fit_transform(Y[train_index,:])
    X_train = X[train_index,:]
    model.fit(X_train, Yr_train)
    
    Yr_test = reducer.transform(Y[test_index,:])
    X_test = X[test_index,:]
    Yr_pred = model.predict(X_test) # , Yr_train)
    r2 = r2_score(Yr_test,  Yr_pred )
    print('r2 test:', r2)
 
    r2_test = r2    
    
    Yr_pred = model.predict(X_train) # , Yr_train)
    r2 = r2_score(Yr_train,  Yr_pred )
    print('r2 train', r2)
    

    Y_submit =  (Y_submit * cnt_blend_submit + reducer.inverse_transform(model.predict(X_submit))) / \
    ( cnt_blend_submit + 1)
    
    cnt_blend_submit += 1
    
    
s = mean_squared_error(Y,Y_oof, squared = False)
print(s)

(614, 20)
Fold 0: 307
r2 test: 0.04486505380880883
r2 train 0.21193333043478044
Fold 1: 307
r2 test: 0.015900175394891778
r2 train 0.21775727573293788
2.081980660485236


In [55]:
Y_submit.shape

(255, 18211)

In [58]:
df_submit = pd.DataFrame(Y_submit, columns = train_df.columns[5:])
df_submit.index.name = 'id'
print( df_submit.shape )
display(df_submit)
df_submit.to_csv('submission.csv')

(255, 18211)


,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
id,,,,,,,,,,,,,,,,,,,,,
0,0.184632,0.198192,0.328456,0.608568,0.346795,0.082029,-0.060384,0.359040,-0.378030,0.430945,...,-0.288571,-0.094996,0.055179,-0.018298,0.311023,0.290961,0.304827,0.310928,-0.264254,0.129146
1,0.155701,0.062785,0.077395,0.026308,0.136985,0.481920,-0.015797,0.148192,0.018365,-0.029024,...,-0.000354,0.050267,0.051659,0.039070,0.189476,0.131311,0.104351,0.037109,0.049731,-0.076314
2,0.346700,0.184278,0.161366,0.100293,0.328118,0.999008,0.006118,0.232142,-0.027576,0.067021,...,-0.069292,0.082356,0.133654,0.200649,0.357414,0.256524,0.182652,0.099959,0.017351,-0.081001
3,-0.001439,-0.002812,0.048068,0.137358,0.079618,-0.082014,-0.045000,0.073377,-0.091452,0.143549,...,-0.032375,-0.067670,-0.042829,-0.044965,0.137725,0.068578,0.111075,0.095033,-0.045690,-0.035634
4,0.102137,0.047725,0.145800,0.123433,0.174148,0.206457,-0.101474,0.191011,-0.083888,0.155294,...,-0.099772,-0.052397,0.020632,-0.023808,0.159020,0.107839,0.155606,0.118234,-0.080964,0.003142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.104983,-0.092289,-0.077584,-0.074771,-0.207447,-0.322679,-0.077362,-0.093349,-0.024482,0.110629,...,-0.066136,-0.066718,-0.103390,-0.168945,-0.071965,-0.092350,0.014509,0.009895,0.047396,-0.025557
251,0.085030,0.018879,0.058391,0.110567,0.103260,0.286532,-0.042078,0.041723,-0.121012,0.260973,...,-0.130305,-0.036041,-0.001706,0.015082,0.157773,0.071284,0.142545,0.122140,0.031942,-0.052038
252,-0.050596,-0.042815,-0.016694,0.061776,-0.063931,-0.194559,-0.052647,-0.009262,-0.089327,0.154116,...,-0.073497,-0.067327,-0.070472,-0.098165,0.035178,-0.014335,0.066729,0.054339,0.022933,-0.022352
